In [1]:
from datetime import datetime

import pandas as pd
import numpy as np

from modules.judge import judge
from modules.info import *
from modules.save2csv import *


In [2]:
entity_label = pd.read_csv(path + 'outputs/1_带标签的主体_' + date + '.csv')
entity_fin = pd.read_csv(path + 'outputs/2_计算好指标的主体_' + date + '.csv')
threshold = pd.read_excel(path + 'rules/22_19绩效长表_230509.xlsx')

In [3]:
threshold['StatisticalCalibre'] = threshold['StatisticalCalibre'].str.replace('企业', '')

In [4]:
entity_label = entity_label[['证券代码', '证券简称', 'ComFeature-GeneralInd-Ind-1', 'ComFeature-GeneralInd-Ind-2', 'ComFeature-GeneralPerform-Scale']]
entity_label['ComFeature-GeneralPerform-Scale'] = entity_label['ComFeature-GeneralPerform-Scale'].fillna('全行业')
entity_label.rename(columns={'证券代码': 'stock_code',
                             '证券简称': 'stock_name'}, inplace=True)

entity_fin.drop(columns=['entity_id', 'uniform_credit', 'industry_whitewash'], inplace=True)
entity_fin = pd.melt(entity_fin, id_vars=['stock_code', 'stock_name', 'entity_name', 'year'])

entity_label_fin = pd.merge(entity_label, entity_fin, on=['stock_code', 'stock_name'])

In [5]:
threshold.rename(columns={'IndustryName': 'ComFeature-GeneralInd-Ind',
                          'StatisticalCalibre': 'ComFeature-GeneralPerform-Scale',
                          'SgnYear': 'year',
                          'indicator_code': 'variable'}, inplace=True)

In [6]:
# 先计算全国
threshold_country = threshold.rename(columns={'ComFeature-GeneralInd-Ind': 'ComFeature-GeneralInd-Ind-1'})

entity_label_fin_country = pd.merge(entity_label_fin, threshold_country,
                                    on=['ComFeature-GeneralInd-Ind-1', 'ComFeature-GeneralPerform-Scale', 'year', 'variable'],
                                    how='left')

conditions = [
    entity_label_fin_country['value'] >= entity_label_fin_country['score_优秀值'],
    (entity_label_fin_country['value'] < entity_label_fin_country['score_优秀值']) & (entity_label_fin_country['value'] >= entity_label_fin_country['score_良好值']),
    (entity_label_fin_country['value'] < entity_label_fin_country['score_良好值']) & (entity_label_fin_country['value'] >= entity_label_fin_country['score_平均值']),
    (entity_label_fin_country['value'] < entity_label_fin_country['score_平均值']) & (entity_label_fin_country['value'] >= entity_label_fin_country['score_较低值']),
    (entity_label_fin_country['value'] < entity_label_fin_country['score_较低值']) & (entity_label_fin_country['value'] >= entity_label_fin_country['score_较差值']),
    entity_label_fin_country['value'] < entity_label_fin_country['score_较差值']
]

choices = [0, 1, 2, 3, 4, 5]
entity_label_fin_country['level_country'] = np.select(conditions, choices, default=0)
# entity_label_fin_country.drop(columns=['ComFeature-GeneralInd-Ind-1', 'ComFeature-GeneralInd-Ind-2', 'ComFeature-GeneralPerform-Scale'], inplace=True)

entity_label_fin_country = entity_label_fin_country[['stock_code', 'stock_name', 'entity_name', 'year', 'variable', 'value', 'level_country']]

entity_label_fin_country


,stock_code,stock_name,entity_name,year,variable,value,level_country
0,000002.SZ,万科A,万科企业股份有限公司,2021,NOTES_AOO_AuditOpinion_idou,NaN,0
1,000002.SZ,万科A,万科企业股份有限公司,2020,NOTES_AOO_AuditOpinion_idou,NaN,0
2,000002.SZ,万科A,万科企业股份有限公司,2019,NOTES_AOO_AuditOpinion_idou,NaN,0
3,000002.SZ,万科A,万科企业股份有限公司,2021,EN_ANNUAL_MarketCapitalization_idou,NaN,0
4,000002.SZ,万科A,万科企业股份有限公司,2020,EN_ANNUAL_MarketCapitalization_idou,NaN,0
...,...,...,...,...,...,...,...
793788,2080369.IB,20甘肃资管小微债,甘肃资产管理有限公司,2020,MinProfitAndProfitAfterDedu,1.254249e+08,0
793789,2080369.IB,20甘肃资管小微债,甘肃资产管理有限公司,2021,Sum3YRDInvestmentToRevenue,1.252072e-03,0
793790,2080369.IB,20甘肃资管小微债,甘肃资产管理有限公司,2020,Sum3YRDInvestmentToRevenue,3.118975e-01,0
793791,2080369.IB,20甘肃资管小微债,甘肃资产管理有限公司,2021,SumMinProfitAndProfitAfterDedu,8.039353e+08,0


In [7]:
# 再计算地方
threshold_local = threshold.rename(columns={'ComFeature-GeneralInd-Ind': 'ComFeature-GeneralInd-Ind-2'})

entity_label_fin_local = pd.merge(entity_label_fin, threshold_local,
                                  on=['ComFeature-GeneralInd-Ind-2', 'year', 'variable'],
                                  how='left')

conditions = [
    entity_label_fin_local['value'] >= entity_label_fin_local['score_优秀值'],
    (entity_label_fin_local['value'] < entity_label_fin_local['score_优秀值']) & (entity_label_fin_local['value'] >= entity_label_fin_local['score_良好值']),
    (entity_label_fin_local['value'] < entity_label_fin_local['score_良好值']) & (entity_label_fin_local['value'] >= entity_label_fin_local['score_平均值']),
    (entity_label_fin_local['value'] < entity_label_fin_local['score_平均值']) & (entity_label_fin_local['value'] >= entity_label_fin_local['score_较低值']),
    (entity_label_fin_local['value'] < entity_label_fin_local['score_较低值']) & (entity_label_fin_local['value'] >= entity_label_fin_local['score_较差值']),
    entity_label_fin_local['value'] < entity_label_fin_local['score_较差值']
]

choices = [0, 1, 2, 3, 4, 5]
entity_label_fin_local['level_local'] = np.select(conditions, choices, default=0)
# entity_label_fin_local.drop(columns=['ComFeature-GeneralInd-Ind-1', 'ComFeature-GeneralInd-Ind-2', 'ComFeature-GeneralPerform-Scale_x', 'ComFeature-GeneralPerform-Scale_y'], inplace=True)

entity_label_fin_local = entity_label_fin_local[['stock_code', 'stock_name', 'entity_name', 'year', 'variable', 'value', 'level_local']]


In [8]:
entity_label_fin_country_local = pd.merge(entity_label_fin_country, entity_label_fin_local)

entity_label_fin_country_local

,stock_code,stock_name,entity_name,year,variable,value,level_country,level_local
0,000002.SZ,万科A,万科企业股份有限公司,2021,NOTES_AOO_AuditOpinion_idou,NaN,0,0
1,000002.SZ,万科A,万科企业股份有限公司,2020,NOTES_AOO_AuditOpinion_idou,NaN,0,0
2,000002.SZ,万科A,万科企业股份有限公司,2019,NOTES_AOO_AuditOpinion_idou,NaN,0,0
3,000002.SZ,万科A,万科企业股份有限公司,2021,EN_ANNUAL_MarketCapitalization_idou,NaN,0,0
4,000002.SZ,万科A,万科企业股份有限公司,2020,EN_ANNUAL_MarketCapitalization_idou,NaN,0,0
...,...,...,...,...,...,...,...,...
793788,2080369.IB,20甘肃资管小微债,甘肃资产管理有限公司,2020,MinProfitAndProfitAfterDedu,1.254249e+08,0,0
793789,2080369.IB,20甘肃资管小微债,甘肃资产管理有限公司,2021,Sum3YRDInvestmentToRevenue,1.252072e-03,0,0
793790,2080369.IB,20甘肃资管小微债,甘肃资产管理有限公司,2020,Sum3YRDInvestmentToRevenue,3.118975e-01,0,0
793791,2080369.IB,20甘肃资管小微债,甘肃资产管理有限公司,2021,SumMinProfitAndProfitAfterDedu,8.039353e+08,0,0


In [9]:
save2csv(entity_label_fin_country_local, '6_绩效触发结果')